In [ ]:
# ! gdown 10TD7sIZAzGGwfTTQttvZlUM7DC6j_CaL
# ! unzstd 21_22_polit.csv.zst
import ast
import re
import torch

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from collections import Counter, defaultdict
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Первичная обработка данных

In [5]:
periods = {
    "BeforeZVO": ('2022-01-23', '2022-02-23'),
    "AfterZVO": ('2022-02-24', '2022-03-24'),
          }

period_data = {key: [] for key in periods.keys()}

# Функция для извлечения всех ссылок из текста
def extract_links(text):
    links = re.findall(r'https://t.me/[^\s]+', text)
    return links if links else None  # Если ссылок нет, возвращаем None

for df in tqdm(pd.read_csv('21_22_polit.csv', chunksize=2_000_000,
                           names=['name', 'time', 'content', 'sourse'], parse_dates=['time'], dtype={'name': str, 'content': str})):
    df['name'] = df['name'].str.split('/').str[0]
    df['content'] = df['content'].fillna('')

    # Добавляем новый столбец 'link' с извлеченными ссылками
    df['link'] = df['content'].apply(extract_links)

    # Оставляем только строки с хотя бы одной ссылкой
    df = df[df['link'].notna()]

    # Обрабатываем каждый период
    for period_name, (start_date, end_date) in periods.items():
        period_df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]
        period_df = period_df.groupby('name').head(1000)
        period_data[period_name].append(period_df)

for period_name, dfs in period_data.items():
  final_df = pd.concat(dfs, ignore_index=True).sort_values(by='time')
  final_df = final_df.iloc[:500_000]
  final_df.to_csv(f'{period_name}.csv', index=False)

print("It's over")

11it [09:05, 49.58s/it]


It's over


In [ ]:
df_after = pd.read_csv("AfterZVO.csv")
df_before = pd.read_csv("BeforeZVO.csv")
df_after['SVO'] = 1
df_before['SVO'] = 0
df = pd.concat([df_after, df_before], ignore_index=True).drop_duplicates().reset_index(drop=True)
df['sourse'] = df['sourse'].apply(ast.literal_eval)

df = df.explode('sourse')
df = df.reset_index(drop=True)
df = df[df['sourse'].astype(str).str.contains('https://t.me/|http://t.me/')]
df.dropna(inplace=True)
df['sourse'] = df['sourse'].str.split('/').str[-2]
df.drop_duplicates(inplace=True)

In [4]:
df

,name,time,content,sourse,link,SVO
0,SBelkovskiy,2022-02-24 00:15:21+00:00,Вот и обыски на Москве обычно начинаются где-т...,news_echo,['https://t.me/news_echo/10813'],1
1,yuzhnocity,2022-02-24 00:46:12+00:00,Во время эвакуации нужно было забирать самое н...,breakingmash,['https://t.me/breakingmash/31480'],1
3,madam_secretar,2022-02-24 00:49:02+00:00,"Лягут как лёгкая бригада. Или побегут, задрав ...",vesparevenge,['https://t.me/madam_secretar/6694'],1
4,madam_secretar,2022-02-24 00:49:02+00:00,"Лягут как лёгкая бригада. Или побегут, задрав ...",madam_secretar,['https://t.me/madam_secretar/6694'],1
6,andyhukka_live,2022-02-24 00:50:32+00:00,Вторжение должно начаться с довольно длительно...,rusbrief,['https://t.me/mozhemobyasnit/11339'],1
...,...,...,...,...,...,...
148583,dtlive,2022-02-22 23:13:22+00:00,Знаю автора несколько лет. Василий был гостем ...,mihalych67,['https://t.me/ukr_leaks/1377'],0
148584,dtlive,2022-02-22 23:13:22+00:00,Знаю автора несколько лет. Василий был гостем ...,ukr_leaks,['https://t.me/ukr_leaks/1377'],0
148585,dtlive,2022-02-22 23:32:05+00:00,«И бесплатно отряд поскакал на врага…» (с)Из п...,OpenUkraine,['https://t.me/OpenUkraine/6120'],0
148586,dtlive,2022-02-22 23:51:48+00:00,"А вот тут уже 400 депутатовТак что, расширять ...",vv_volodin,['https://t.me/vv_volodin/312'],0


# Сентимент

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch
import numpy as np
from tqdm import tqdm
from collections import Counter

In [8]:
# Конфигурация
MODEL_NAME = "blanchefort/rubert-base-cased-sentiment"
BATCH_SIZE = 8
MAX_LENGTH = 512
LABELS = ["Negative", "Neutral", "Positive"]

class TextDataset(Dataset):
    """Датасет для обработки текстов батчами"""
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

class SentimentAnalyzer:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        self.model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(self.device)
        self.model.eval()

    def analyze(self, texts: list, batch_size: int = BATCH_SIZE) -> list:
        """Анализ сентимента для списка текстов"""
        dataset = TextDataset(texts)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        results = []
        for batch in tqdm(dataloader):

            inputs = self.tokenizer(
                batch,
                max_length=MAX_LENGTH,
                padding=True,
                truncation=True,
                return_tensors="pt"
            ).to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)

            probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()

            for prob in probs:
                results.append({
                    "sentiment": LABELS[np.argmax(prob)],
                })


        return results

analyzer = SentimentAnalyzer()

# Тестовые данные
texts = df['content'].to_list()
# Анализ текстов
results = analyzer.analyze(texts)


100%|██████████| 13365/13365 [12:05:21<00:00,  3.26s/it]     


In [ ]:
df = df.reset_index(drop=True)
df['sentiment'] = pd.DataFrame(results)['sentiment']
df

,name,time,content,sourse,link,SVO,sentiment
0,SBelkovskiy,2022-02-24 00:15:21+00:00,Вот и обыски на Москве обычно начинаются где-т...,news_echo,['https://t.me/news_echo/10813'],1,Positive
1,yuzhnocity,2022-02-24 00:46:12+00:00,Во время эвакуации нужно было забирать самое н...,breakingmash,['https://t.me/breakingmash/31480'],1,Neutral
3,madam_secretar,2022-02-24 00:49:02+00:00,"Лягут как лёгкая бригада. Или побегут, задрав ...",vesparevenge,['https://t.me/madam_secretar/6694'],1,Positive
4,madam_secretar,2022-02-24 00:49:02+00:00,"Лягут как лёгкая бригада. Или побегут, задрав ...",madam_secretar,['https://t.me/madam_secretar/6694'],1,Positive
6,andyhukka_live,2022-02-24 00:50:32+00:00,Вторжение должно начаться с довольно длительно...,rusbrief,['https://t.me/mozhemobyasnit/11339'],1,Positive
...,...,...,...,...,...,...,...
148583,dtlive,2022-02-22 23:13:22+00:00,Знаю автора несколько лет. Василий был гостем ...,mihalych67,['https://t.me/ukr_leaks/1377'],0,NaN
148584,dtlive,2022-02-22 23:13:22+00:00,Знаю автора несколько лет. Василий был гостем ...,ukr_leaks,['https://t.me/ukr_leaks/1377'],0,NaN
148585,dtlive,2022-02-22 23:32:05+00:00,«И бесплатно отряд поскакал на врага…» (с)Из п...,OpenUkraine,['https://t.me/OpenUkraine/6120'],0,NaN
148586,dtlive,2022-02-22 23:51:48+00:00,"А вот тут уже 400 депутатовТак что, расширять ...",vv_volodin,['https://t.me/vv_volodin/312'],0,NaN


# Идеологии

In [42]:
categories = {
    "консервативная": {
        # ценности
        "традиции", "традиционные", "семья", "патриот", "патриотизм", "родина", "отечество", "нация", "порядок",
        "духовность", "идентичность", "мораль", "скрепы", "вера", "уклад", "русский мир", "ценности", "религия",
        # оценочные
        "гниль", "либераха", "толерасты", "западло", "вырождение", "дегенераты",
        # мемы/разговорные
        "мама и папа", "настоящие мужики", "гейропа", "пиндосы", "пендосия", "боженька", "традишнл вайф", "ватник"
    },

    "либеральная": {
        # ценности
        "свобода", "демократия", "права человека", "гуманизм", "толерантность", "плюрализм", "реформы", "сми",
        "независимость", "свободный рынок", "либеральная экономика", "активизм", "волонтёр", "гражданин",
        # лексика по меньшинствам
        "лгбт", "куир", "прайд", "трансгендер", "идентичность", "гендер", "дискриминация",
        # мемы/разговорные
        "либераха", "зожник", "токсичная маскулинность", "cancel culture", "феминистки", "толерасты", "винить систему",
        "фактчек", "деколонизация", "репрессии", "путинрежим", "диктатор", "авторитаризм", "режим"
    },

    "коммунистическая": {
        # ценности
        "труд", "пролетариат", "эксплуатация", "социализм", "революция", "коммунизм", "интернационал",
        "рабочие", "завод", "буржуазия", "госплан", "товарищ", "национализация", "госкапитализм",
        # критика капитала
        "олигархи", "капиталюги", "буржуи", "кулаки", "эксплуататоры", "чинуши", "жирные коты",
        # мемы/разговорные
        "совок", "вперёд, товарищи!", "пролетарии всех стран", "всё — народу", "красные", "ленинисты",
        "сталинисты", "партейцы", "буржуй", "паразиты", "всех раскулачить", "буржуазная демократия"
    }
}

In [43]:
category_names = list(categories.keys())
category_keywords = [' '.join(values) for values in categories.values()]

In [44]:
model = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device='cpu'
)

category_embeddings = model.encode(category_keywords)

In [45]:
def predict_topics(texts, model, category_embeddings, category_names, batch_size=32):
    topics = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch)
        similarities = cosine_similarity(batch_embeddings, category_embeddings)
        max_indices = np.argmax(similarities, axis=1)
        batch_topics = [category_names[idx] for idx in max_indices]
        topics.extend(batch_topics)

    return topics


In [ ]:
texts = df['content'].astype(str).tolist()
df['topic'] = predict_topics(texts, model, category_embeddings, category_names)

100%|██████████| 3342/3342 [19:47<00:00,  2.81it/s]


In [52]:
df

,name,time,content,sourse,link,SVO,sentiment,topic
0,SBelkovskiy,2022-02-24 00:15:21+00:00,Вот и обыски на Москве обычно начинаются где-т...,news_echo,['https://t.me/news_echo/10813'],1,Positive,либеральная
1,yuzhnocity,2022-02-24 00:46:12+00:00,Во время эвакуации нужно было забирать самое н...,breakingmash,['https://t.me/breakingmash/31480'],1,Neutral,консервативная
2,madam_secretar,2022-02-24 00:49:02+00:00,"Лягут как лёгкая бригада. Или побегут, задрав ...",vesparevenge,['https://t.me/madam_secretar/6694'],1,Positive,консервативная
3,madam_secretar,2022-02-24 00:49:02+00:00,"Лягут как лёгкая бригада. Или побегут, задрав ...",madam_secretar,['https://t.me/madam_secretar/6694'],1,Positive,консервативная
4,andyhukka_live,2022-02-24 00:50:32+00:00,Вторжение должно начаться с довольно длительно...,rusbrief,['https://t.me/mozhemobyasnit/11339'],1,Positive,консервативная
...,...,...,...,...,...,...,...,...
106909,dtlive,2022-02-22 23:13:22+00:00,Знаю автора несколько лет. Василий был гостем ...,mihalych67,['https://t.me/ukr_leaks/1377'],0,Positive,консервативная
106910,dtlive,2022-02-22 23:13:22+00:00,Знаю автора несколько лет. Василий был гостем ...,ukr_leaks,['https://t.me/ukr_leaks/1377'],0,Positive,консервативная
106911,dtlive,2022-02-22 23:32:05+00:00,«И бесплатно отряд поскакал на врага…» (с)Из п...,OpenUkraine,['https://t.me/OpenUkraine/6120'],0,Negative,консервативная
106912,dtlive,2022-02-22 23:51:48+00:00,"А вот тут уже 400 депутатовТак что, расширять ...",vv_volodin,['https://t.me/vv_volodin/312'],0,Positive,либеральная


In [ ]:
df.to_csv("final_data.csv")